In [1]:
%load_ext autoreload
%autoreload 2

# Counterfactual explanations for the GSMarena dataset

### Installations

Run the following cells to download the repository and install the required packages.

This is needed only on Google Colab, if you are running this notebook locally, you can skip this step.

In [2]:
COLAB_ENV = "google.colab" in str(get_ipython()) if hasattr(__builtins__, "__IPYTHON__") else False
print(f"Currently {'not ' if not COLAB_ENV else ''}running in a Colab environment")

Currently not running in a Colab environment


In [3]:
if COLAB_ENV:
    !mkdir my_repo
    !git clone https://github.com/Valendrew/counterfactual-explanations.git my_repo
    %cd my_repo
    !pip install -r requirements.txt -q

### Import of the libraries

In [4]:
import numpy as np
import pandas as pd

In [5]:
def set_default_randomness(seed: int):
    rng = np.random.RandomState(seed)
    return rng


SEED = 42
rng = set_default_randomness(SEED)

## Evaluation on multi-class classification

In [17]:
def change_dataframe(df, label: int):
    df = df.copy()
    df = df.T
    df.insert(0, "misc_price", label)
    return df

In [39]:
concat_df = pd.DataFrame()
for cf_label, file_name in zip([1, 0, 2, 1], ["low_to_medium", "medium_to_low", "medium_to_high", "high_to_medium"]):
    print(f"CF label: {cf_label}")
    print(f"File name: {file_name}")
    df = pd.read_csv(f"data/counterfactual/{file_name}.old.csv", index_col=0)
    df = change_dataframe(df, cf_label)

    concat_df = pd.concat([concat_df, df], axis=0)

# Cast numerical features to float
cat_features = ["display_resolution", "network_technology"]
num_features = concat_df.columns.drop(cat_features)
concat_df[num_features] = concat_df[num_features].astype(float)

# Round discrete features to integers
round_columns = num_features.drop(["misc_price", "misc_price_min", "misc_price_max", "display_size", "battery"])
concat_df[round_columns] = concat_df[round_columns].round(0).astype(int)

# Round continuous features to 2 decimals
round_columns = ["display_size", "battery"]
concat_df[round_columns] = concat_df[round_columns].round(2)

# Save to csv
concat_df.to_csv("data/counterfactual/multiclass-two-objs.csv")

CF label: 1
File name: low_to_medium
CF label: 0
File name: medium_to_low
CF label: 2
File name: medium_to_high
CF label: 1
File name: high_to_medium


## Evaluation on binary-class classification